<a href="https://colab.research.google.com/github/ryouy/election2026/blob/main/NHKelection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pathlib import Path
import shutil

from google.colab import drive
drive.mount('/content/drive')

SRC_DIR = Path("/content/drive/MyDrive/NHK_enquete_2026/htmls")
IN_DIR = Path("/content/all")

# 出力先を作る
IN_DIR.mkdir(parents=True, exist_ok=True)

# 中身をすべてコピー
for item in SRC_DIR.iterdir():
    dest = IN_DIR / item.name
    if item.is_dir():
        shutil.copytree(item, dest, dirs_exist_ok=True)
    else:
        shutil.copy2(item, dest)

print("コピー完了:", IN_DIR)


Mounted at /content/drive
コピー完了: /content/all


In [ ]:
from pathlib import Path
import re
import pandas as pd
from bs4 import BeautifulSoup

# ========= 入出力 =========
IN_DIR = Path("/content/all")   # html / txt が入っているフォルダ
OUT_FINAL = Path("/content/all_candidate.csv")
OUT_UNMATCHED = Path("/content/final_unmatched.csv")

# ========= 対応表（選択式だけの Q1～Q22） =========
CHOICE_MAP = {
    1: ["妥当だ","妥当ではない","どちらともいえない","回答しない"],
    2: ["大いに評価する","ある程度評価する","あまり評価しない","全く評価しない","回答しない"],
    3: ["よい影響がある","悪い影響がある","特に影響はない","回答しない"],
    4: ["賛成","反対","どちらともいえない","回答しない"],
    5: ["日米同盟を重視し、アメリカとの連携をさらに強化すべき",
        "日米同盟だけに依存せず、国際協調を重視すべき",
        "日米同盟を解消すべき","どちらともいえない","回答しない"],
    6: ["より強い姿勢で臨むべき","関係改善をより重視すべき","今のままでよい","その他","回答しない"],
    7: ["さらに強化すべき","今の政府の方針でよい","強化は必要だが防衛費は抑制すべき","強化する必要はない","回答しない"],
    8: ["5類型は撤廃すべき","5類型を維持すべき","防衛装備品の移転そのものをやめるべき","その他","回答しない"],
    9: ["維持すべき","見直すべき","どちらともいえない","回答しない"],
    10:["禁止すべき","禁止せず規制を強化すべき","今のまま維持すべき","回答しない"],
    11:["すみやかに削減すべきだ","時間をかけて議論すべきだ","定数削減をする必要はない","回答しない"],
    12:["高齢者の負担を増やすべき","現役世代の負担を増やすべき","所得が多い人の負担を増やすべき",
        "給付を大幅に抑制して、負担を増やさないようにすべき","その他","回答しない"],
    13:["大いに評価する","ある程度評価する","あまり評価しない","全く評価しない","回答しない"],
    14:["需要に応じて生産すべき","増産すべき","どちらともいえない","その他","回答しない"],
    15:["高めるべき","今の程度でよい","下げるべき","ゼロにすべき","回答しない"],
    16:["さらに積極的に受け入れるべき","今の程度でよい","受け入れを抑制すべき","受け入れるべきではない","回答しない"],
    17:["大いに評価する","ある程度評価する","あまり評価しない","全く評価しない","回答しない"],
    18:["夫婦が希望すれば結婚前の姓を名乗れる「選択的夫婦別姓」を導入すべき",
        "「夫婦同姓」を維持し、旧姓の通称使用を認める法制度を拡充すべき",
        "今の「夫婦同姓」の法律のままでよい","その他","回答しない"],
    19:["賛成","反対","どちらともいえない","回答しない"],
    20:["必要がある","必要はない","どちらともいえない","回答しない"],
    21:["賛成","反対","どちらともいえない","回答しない"],
    22:["女性天皇も女系天皇も「賛成」",
        "女性天皇は「賛成」。女系天皇は「反対」",
        "女性天皇も女系天皇も「反対」",
        "回答しない"],
}

# ========= 正規化 =========
def norm(s: str) -> str:
    s = str(s).strip()
    s = re.sub(r"[ \t　]+", " ", s)
    s = s.replace("、", "。")
    s = s.rstrip("。")
    return s

def normalize_name(name: str) -> str:
    s = str(name).strip()
    s = s.replace("　", " ")
    s = re.sub(r"\s+", " ", s)
    return s

# choice_lookup: qid -> { normalized_choice_text: code }
choice_lookup = {}
for qid, choices in CHOICE_MAP.items():
    d = {}
    for idx, ch in enumerate(choices, start=1):
        d[norm(ch)] = idx
    choice_lookup[qid] = d

# ========= 記述式判定 =========
def is_free_text_question(q_text: str) -> bool:
    q = q_text.strip()
    if "回答の理由" in q:
        return True
    if "さらにどんなことに力を入れるべき" in q:
        return True
    if "具体的に" in q and "お答えください" in q:
        return True
    return False

# ========= 正規表現 =========
Q_LINE_RE = re.compile(r"^Q[:：]\s*(.*)$")
NAME_ANS_RE = re.compile(r"^(.+?)(?:　+|\s{2,})(.+)$")

# ========= HTML → テキスト =========
def html_to_text(html: str) -> str:
    soup = BeautifulSoup(html, "html.parser")
    for br in soup.find_all("br"):
        br.replace_with("\n")
    return soup.get_text()

# ========= 1ファイル処理 =========
def parse_one_file(path: Path):
    raw = path.read_text(encoding="utf-8")

    if path.suffix.lower() == ".html":
        text = html_to_text(raw)
    else:
        text = raw

    lines = text.splitlines()

    out_q = 0
    data = {}
    unmatched = []

    for raw_line in lines:
        s = raw_line.strip()
        if not s:
            continue

        mQ = Q_LINE_RE.match(s)
        if mQ:
            q_text = mQ.group(1).strip()
            if is_free_text_question(q_text):
                continue
            out_q += 1
            continue

        m = NAME_ANS_RE.match(s)
        if m and (1 <= out_q <= 22):
            name = normalize_name(m.group(1))
            ans = m.group(2).strip()

            data.setdefault(name, {})
            key = norm(ans)
            code = choice_lookup.get(out_q, {}).get(key, "")

            if code == "":
                unmatched.append((path.name, name, f"Q{out_q}", ans))
            else:
                data[name][out_q] = code

    rows = []
    for name, ansmap in data.items():
        num = path.stem
        url = f"https://news.web.nhk/senkyo-data/database/shugiin/2026/survey/k/{num}.html"
        row = {"氏名": name}
        for q in range(1, 23):
            row[f"Q{q}"] = ansmap.get(q, "")
        rows.append(row)

    df = pd.DataFrame(rows).sort_values("氏名").reset_index(drop=True)
    um = pd.DataFrame(unmatched, columns=["ファイル名","氏名","Q","原文回答"])
    return df, um

# ========= 全ファイル処理 =========
all_files = sorted(list(IN_DIR.glob("*.html")) + list(IN_DIR.glob("*.txt")))
if not all_files:
    raise FileNotFoundError(f"{IN_DIR} に html/txt が見つかりません")

dfs = []
ums = []

for p in all_files:
    df_one, um_one = parse_one_file(p)
    dfs.append(df_one)
    ums.append(um_one)

final_df = pd.concat(dfs, ignore_index=True)

q_cols = [f"Q{i}" for i in range(1, 23)]
final_df = final_df[["氏名"] + q_cols]

final_df.to_csv(OUT_FINAL, index=False, encoding="utf-8-sig")
print("final 保存:", OUT_FINAL, "rows=", len(final_df))

final_um = pd.concat(ums, ignore_index=True)
final_um.to_csv(OUT_UNMATCHED, index=False, encoding="utf-8-sig")
print("unmatched 保存:", OUT_UNMATCHED, "rows=", len(final_um))

final_df.head()


final 保存: /content/all_candidate.csv rows= 1038
unmatched 保存: /content/final_unmatched.csv rows= 13


,氏名,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,...,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Q20,Q21,Q22
0,出口 洋介,2,4,2,1,3,2,4,3,1,...,4,2,4,2,4,1,1,2,2,1
1,田村 憲久,1,1,1,3,1,1,1,1,3,...,1,1,1,5,1,2,2,1,1,3
2,福森 和歌子,2,2,4,1,2,4,3,2,1,...,2,3,3,2,3,1,1,3,3,4
3,下野 幸助,2,3,2,1,2,2,3,4,1,...,2,4,3,1,4,1,1,1,1,4
4,川崎 秀人,1,2,1,2,1,1,1,1,1,...,2,1,1,2,2,1,2,1,1,3


In [ ]:
# ==== パスを自分のDriveに合わせて変更 ====
master_csv = Path("/content/drive/MyDrive/yomiuri_enquete_2026/all/_all_candidates.csv")  # 先ほど作ったCSV
survey_csv = Path("/content/all_candidate.csv")          # 立候補者マスタ
out_csv    = Path("/content/all_candidates.csv")



def normalize_name(name: str) -> str:
    if pd.isna(name):
        return ""
    s = str(name).strip()
    s = s.replace("　", " ")
    s = re.sub(r"\s+", " ", s)
    return s

def normalize_col(col: str) -> str:
    # 列名の前後スペース、全角スペース、見えない文字を掃除
    s = str(col).strip().replace("　", " ")
    s = re.sub(r"\s+", " ", s)
    return s

# ==== 読み込み ====
df = pd.read_csv(survey_csv, dtype=str)
master = pd.read_csv(master_csv, dtype=str)

# ==== 列名を正規化して確認 ====
master.columns = [normalize_col(c) for c in master.columns]
print("master columns:", list(master.columns))

# ==== 必須：氏名列を特定（'氏名' が無い場合もあるので柔軟に） ====
# よくある候補：'氏名', '名前', '候補者', '候補者名'
name_col_candidates = ["氏名", "名前", "候補者", "候補者名"]
name_col = next((c for c in name_col_candidates if c in master.columns), None)
if name_col is None:
    raise ValueError(f"氏名列が見つかりません。master columns={list(master.columns)}")

# ==== 追加したい列（存在するものだけ拾う） ====
# 列名ゆれ候補も吸収
col_aliases = {
    "グループ": ["グループ", "group", "ブロック", "選挙区", "区分"],
    "年齢": ["年齢", "齢", "age"],
    "政党": ["政党", "党", "所属", "party"],
}

picked = {}
for std_name, aliases in col_aliases.items():
    hit = next((c for c in aliases if c in master.columns), None)
    picked[std_name] = hit

print("picked columns mapping:", picked)

# ここでどれも見つからない場合は、masterの列名が全然違うので
# printされた master columns を見て aliases を足すのが最短です。

# ==== join key 作成 ====
df["氏名_key"] = df["氏名"].map(normalize_name)
master["氏名_key"] = master[name_col].map(normalize_name)

# ==== masterから必要列を取り出して標準名にリネーム ====
use_cols = ["氏名_key"] + [c for c in picked.values() if c is not None]
master_sel = master[use_cols].copy()

rename_map = {v: k for k, v in picked.items() if v is not None}
master_sel = master_sel.rename(columns=rename_map)

# 同姓同名重複があるときは、最初の1件を採用（必要ならここを調整）
master_sel = master_sel.drop_duplicates("氏名_key")

# ==== merge ====
merged = df.merge(master_sel, on="氏名_key", how="left")

# ==== 列順：グループ/氏名/年齢/政党 を前に ====
front = [c for c in ["グループ", "氏名", "年齢", "政党"] if c in merged.columns]
q_cols = [c for c in merged.columns if re.fullmatch(r"Q\d+", c)]
rest = [c for c in merged.columns if c not in front + q_cols + ["氏名_key"]]
merged = merged[front + q_cols + rest]

# ==== 保存 ====
merged.to_csv(out_csv, index=False, encoding="utf-8-sig")
print("保存しました:", out_csv)

# ==== 突合できなかった氏名チェック ====
# "グループ" が無い場合もあるので、存在する先頭列で判定
check_col = "グループ" if "グループ" in merged.columns else (front[0] if front else None)
if check_col:
    unmatched = merged[merged[check_col].isna()][["氏名"]].drop_duplicates()
    if len(unmatched):
        print("\n--- マスタに見つからなかった氏名（表記ゆれ等）---")
        display(unmatched)
    else:
        print("\n全員マスタに突合できました。")
else:
    print("\n突合は実行しました（チェック用の列が無いので未突合一覧は省略）。")


master columns: ['グループ', '氏名', '年齢', '政党', 'Q1-1', 'Q1-2', 'Q1-3', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q9-1', 'Q9-2', 'Q9-3', 'Q9-4', 'Q9-5', 'Q9-6', 'Q10', 'Q11', 'Q12', 'Q13', 'Q14', 'Q15', 'Q16', 'Q17', 'Q18', 'Q19', 'Q20', 'Q21', 'Q22', 'Q23', 'Q24-1', 'Q24-2', 'Q24-3', 'Q24-4', 'Q24-5', 'Q24-6', 'Q24-7', 'Q24-8', 'Q24-9', 'Q24-10', 'Q24-11', 'Q25-1', 'Q25-2', 'Q25-3', 'Q25-4', 'Q25-5', 'Q25-6', 'Q25-7', 'Q25-8', 'Q25-9', 'Q25-10', 'URL', '_status']
picked columns mapping: {'グループ': 'グループ', '年齢': '年齢', '政党': '政党'}
保存しました: /content/all_candidates.csv

--- マスタに見つからなかった氏名（表記ゆれ等）---


,氏名
41,東アジアの安保環境が日々厳しさを増す中、
63,山口 壯
67,舩川 治郎
99,消費税を5％に減税
121,高木 宏壽
140,在留資格、帰化厳格化、土地取得監視強化は一定評価
152,中谷 めぐ
177,安藤 じゅん子
183,齋藤 健
213,三ツ林 裕己


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

from pathlib import Path
import pandas as pd
import re

# ===== 入力 =====
SRC = Path("/content/all_candidates.csv")

# ===== 出力先（Drive） =====
BASE_DIR = Path("/content/drive/MyDrive/NHK_enquete_2026")
PREF_DIR = BASE_DIR / "prefecture"

BASE_DIR.mkdir(parents=True, exist_ok=True)
PREF_DIR.mkdir(parents=True, exist_ok=True)

# ===== util =====
def safe_filename(s: str) -> str:
    s = "" if pd.isna(s) else str(s)
    s = s.strip()
    s = s.replace("　", " ")
    s = re.sub(r"\s+", " ", s)
    s = re.sub(r'[\\/:*?"<>|]+', "_", s)
    s = s.replace(" ", "_")
    s = re.sub(r"_+", "_", s)
    return s if s else "unknown"

# ===== 読み込み =====
df = pd.read_csv(SRC, dtype=str)

# 列名の掃除
df.columns = [str(c).strip().replace("　", " ") for c in df.columns]

# 必須列チェック
required = ["グループ", "氏名", "年齢", "政党"]
missing = [c for c in required if c not in df.columns]
if missing:
    raise ValueError(f"必要な列が見つかりません: {missing}\n実際の列: {list(df.columns)}")

# 軽い正規化
df["グループ"] = df["グループ"].fillna("unknown").astype(str).str.strip()
df["氏名"] = (
    df["氏名"]
    .fillna("")
    .astype(str)
    .str.strip()
    .str.replace("　", " ", regex=False)
    .str.replace(r"\s+", " ", regex=True)
)
df["年齢"] = df["年齢"].fillna("").astype(str).str.strip()
df["政党"] = df["政党"].fillna("unknown").astype(str).str.strip()

# ===== 1) 全員分 =====
all_out = BASE_DIR / "all_candidates_plus.csv"
df.to_csv(all_out, index=False, encoding="utf-8-sig")
print("Saved:", all_out)

# ===== 2) 都道府県（グループ）別 =====
df_pref = df.sort_values(["グループ", "氏名"], kind="stable")
for pref, gdf in df_pref.groupby("グループ", dropna=False):
    fname = safe_filename(pref) + ".csv"
    out_path = PREF_DIR / fname
    gdf.to_csv(out_path, index=False, encoding="utf-8-sig")

print("Saved prefecture files in:", PREF_DIR)

# ===== 3) 政党別（1ファイル、多い順） =====
party_counts = (
    df["政党"]
    .fillna("unknown")
    .value_counts()
    .reset_index()
)
party_counts.columns = ["政党", "人数"]

# 政党の並び順（多い順）をキーにしてソート
df_party_sorted = df.merge(party_counts, on="政党", how="left")
df_party_sorted = df_party_sorted.sort_values(
    ["人数", "政党", "氏名"],
    ascending=[False, True, True],
    kind="stable"
).drop(columns=["人数"])

party_out = BASE_DIR / "_all_candidates_party.csv"
df_party_sorted.to_csv(party_out, index=False, encoding="utf-8-sig")

print("Saved party file:", party_out)

print("\nDone. Output folder:", BASE_DIR)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Saved: /content/drive/MyDrive/NHK_enquete_2026/all_candidates_plus.csv
Saved prefecture files in: /content/drive/MyDrive/NHK_enquete_2026/prefecture
Saved party file: /content/drive/MyDrive/NHK_enquete_2026/_all_candidates_party.csv

Done. Output folder: /content/drive/MyDrive/NHK_enquete_2026
